# Empirical Asset Pricing - PS4

Maximilian Huber

## Task 1
Let me load the data into an array of DataFrames:

In [45]:
using CSV, DataFrames, Query, Plots, Optim; gr();

I load the data, drop rows with a missing value and split the sample in to the two managers.

In [46]:
data = (CSV.read("./Data/PS4data.csv", delim=',', nullable=true,
        types=[String, String, Int64, Int64, Int64, Float64, Float64, Int64, Float64, Float64, Float64, Float64, Int64, Float64, Float64, Float64, Float64]))
dropmissing!(data)

for col in names(data)
   data[col] = Missings.coalesce.(data[col], 0)
end

DFA = data[data[:mgrno] .== 23000, 4:end]
VAN = data[data[:mgrno] .== 90457, 4:end];

### Preliminaries for (a)
Let me try to understand the data set. Since $\sum_{n=0}^N w_i(n) = 1$, $\sum_{n=1}^N w_i(n) = 1 - w_i(0)$, and hence $w_i(0) \Big(1 + \frac{\sum_{n=1}^N w_i(n)}{w_i(0)}\Big) = 1$. Therefore, cash holdings (or whatever the outside asset is) are:

In [47]:
w_DFA0 = 1 / (1 + sum(DFA[:rweight]))

0.10284923721109153

In [48]:
w_VAN0 = 1 / (1 + sum(VAN[:rweight]))

0.1383314470907214

In [49]:
DFA[:weight] = DFA[:rweight] * w_DFA0
VAN[:weight] = VAN[:rweight] * w_VAN0;

Does one fund invest in some assets that the other fund does not? I.e. $\mathcal{N}_{VAN} = \mathcal{N}_{DFA}$?

In [50]:
[length(DFA[:permno]), length(VAN[:permno])] .- length(union(DFA[:permno], VAN[:permno]))

2-element Array{Int64,1}:
 -229
  -78

No, and neither is a superset of the other!

I assume that I can infere $\mathcal{N}_{i}$ just by looking at the current portfolio. This would lead to equal estimates in (b) and (f), I think.

### (a)

In [51]:
active_share(W) = 1/2 * sum(abs.(W[:weight] .- exp.(W[:LNme]) ./ sum(exp.(W[:LNme]))))

active_share(DFA)

0.4535664834349477

In [52]:
active_share(VAN)

0.09184197495623975

DFA is much more active than Vanguard! The latter offers more products that track market-cap-weighted indices.

### (b) - (e)
I loosely follow the Bruce Hansen's [textbook](https://www.ssc.wisc.edu/~bhansen/econometrics/Econometrics.pdf) for the GMM estimation, except I alter some notation a bit. GMM solves:
$$\underset{\theta}{\min}Q_n(\theta)$$
optimizes $Q(\theta) = \frac{1}{2} \, g_n(\theta)' \, \mathcal{W} \, g_n(\theta)$ where $g_n(\theta) = \frac{1}{n}\sum_{t=1}^n g(w_t;\theta)$

In the just-identified IV-GMM case $g(w;\theta)=\Big(w_{lhs} - w_{reg}'\cdot\theta\Big)\cdot w_{iv}\in\mathbb{R}^k$, where $k$ is the number of regressors and also instrumental variables.

In my application the difference between regressors and instruments is just swapping out $LNme$ with $IVme$.

Since I have to do an non-linear GMM estimation in (f) I do not follow Hansen's simple derivations of the GMM in linear models, but set up the optimizition problem generically, using a barrier method optimizer with L-BFGS in the inner problem and auto-differentiation.

The main issue is the correct covariance estimation for the estimated $\hat\theta$.

$$\hat\Omega=\frac{1}{n}\sum_t g(w_t;\tilde\theta)\,g(w_t;\tilde\theta)'$$

No matter whether the constraint is binding or not, the efficient weighting matrix is $\mathcal{W} = \hat\Omega^{-1}$, see BH chapter 12.14 "Restricted GMM".

If $\hat \theta$ is unconstrained, then $\hat V_\theta = \Big(\hat G'\hat\Omega^{-1}\hat G\Big)^{-1}$, where $\hat G = \frac{1}{n}\sum_t \frac{\partial g}{\partial\theta'} (w_t;\hat \theta)$.

But $\hat \theta$ is fulfilling the restriction with equality, I need to correct for that: $\hat V_{\theta,constr} = \hat V_\theta - \hat V_\theta R \Big(R'\hat V_\theta R\Big)^{-1} R' \hat V_\theta$, where R is Jacobian of the restriction $r(\theta) = c$.

In [53]:
#operates on a single observation, w[1] = lhs, w[2:k] = regressors
function g(w, θ)
    k = ceil(Int64, length(w)/2)
    (w[1] - w[2:k]' * θ) * w[k+1:end]
end

function gn(w, θ)
    avg_g = zeros(length(θ))
    for t in 1:size(w, 1)
        avg_g += g(w[t, :], θ)
    end
    
    return avg_g/size(w, 1)
end

function gn_wrapper(W)
    N = size(W, 1)
    w = hcat(W[:LNrweight], 
        Matrix(W[[:LNme, :LNbe, :profit, :Gat, :divA_be, :beta]]), ones(N),
        Matrix(W[[:IVme, :LNbe, :profit, :Gat, :divA_be, :beta]]), ones(N))
    return θ -> gn(w, θ)
end

function Qn(θ, gn_wrapped)
    1/2 * (gn_wrapped(θ)' * gn_wrapped(θ))[1]
end

function Qn(θ, gn_wrapped, Ω)
    1/2 * (gn_wrapped(θ)' * Ω * gn_wrapped(θ))[1]
end

function Ωhat(W, θ)
    N = size(W, 1)
    w = hcat(W[:LNrweight], 
        
        Matrix(W[[:LNme, :LNbe, :profit, :Gat, :divA_be, :beta]]), ones(N),
        Matrix(W[[:IVme, :LNbe, :profit, :Gat, :divA_be, :beta]]), ones(N))
    
    result = zeros(Float64, length(θ), length(θ))
    for i in 1:N
        result .+= g(w[i,:], θ) * g(w[i,:], θ)'
    end
    return result/N
end

Ωhat (generic function with 1 method)

In [54]:
function eff_IV_GMM(W)
    #first stage
    initial_θ = 0.5 * ones(7)
    lower = -Inf * ones(length(initial_θ))
    upper = vcat(1, Inf * ones(length(initial_θ) - 1))
    
    gn_wrapped = gn_wrapper(W)
    obj = OnceDifferentiable(θ -> Qn(θ, gn_wrapped), initial_θ; autodiff = :forward)
    first_θ = Optim.minimizer(optimize(obj, initial_θ, lower, upper, Fminbox{LBFGS}()))

    #second stage
    obj = OnceDifferentiable(θ -> Qn(θ, gn_wrapped, Ωhat(W, first_θ)), first_θ; autodiff = :forward)
    obj.df(first_θ, first_θ)
    second_θ = Optim.minimizer(optimize(obj, first_θ, lower, upper, Fminbox{LBFGS}()))
    
    #asymptotic variance estimation
    G = ForwardDiff.jacobian(gn_wrapped, second_θ)
    Vhat = (G' * Ωhat(W, second_θ)^(-1) * G)^(-1)

    #correction in case of binding constraint
    if second_θ[1] ≈ 1
        R = vcat(1, zeros(length(initial_θ) - 1))
        Vhat = Vhat - Vhat*R * (R'*Vhat*R)^(-1) * R' * Vhat
    end
    
    #return point estimates and sample std errors
    return [second_θ, Vhat/size(W, 1)] 
end

eff_IV_GMM (generic function with 1 method)

In [55]:
table = DataFrame(); table[:coefficient] = [:LNme, :LNbe, :profit, :Gat, :divA_be, :beta, :constant]; 
result = eff_IV_GMM(VAN)
table[:VAN_θ] = result[1]; table[:VAN_θ_stderr] = sqrt.(diag(result[2]))
result = eff_IV_GMM(DFA)
table[:DFA_θ] = result[1]; table[:DFA_θ_stderr] = sqrt.(diag(result[2]))

table

,coefficient,VAN_θ,VAN_θ_stderr,DFA_θ,DFA_θ_stderr
1,LNme,1.0,0.0,0.459876,0.06725
2,LNbe,0.15846,0.016577,0.293579,0.0643384
3,profit,-1.11222,0.0996191,1.47946,0.152318
4,Gat,0.89921,0.118206,-1.0244,0.197911
5,divA_be,32.2135,1.76347,-3.15368,0.783229
6,beta,0.43946,0.0404025,0.360101,0.0503239
7,constant,-17.5322,0.0990471,-12.5887,0.124217


<b>Vanguard</b> is not exactly an index fund, but the $1.0$ coefficient on market equity point into this direction. Other factors are not very important. Vanguard does not seem inelastic.

<b>DFA</b> has a much lower coefficient on market equity, but coefficients on profit, Gat, divA, and beta are much higher in magnitude. DFA does not seem like an index fund, but also not like a value fund, because then the coefficients on market equity and book equity should be the same value with opposing signs. It has a less elastic demand than Vanguard.

### (f)

In [11]:
function g(w, θ)
    k = ceil(Int64, length(w)/2)
    
    ε = exp(w[1]) / exp(w[2:k]' * θ)
    ε * w[k+1:end]
end

g (generic function with 1 method)

In [18]:
W = DFA
#first stage
    initial_θ = 0.5 * ones(7)
    lower = -Inf * ones(length(initial_θ))
    upper = vcat(1, Inf * ones(length(initial_θ) - 1))
    
    gn_wrapped = gn_wrapper(W)
    obj = OnceDifferentiable(θ -> -Qn(θ, gn_wrapped), initial_θ; autodiff = :forward)
@time    results = optimize(obj, initial_θ, lower, upper, Fminbox{LBFGS}(), 
    optimizer_o = Optim.Options(g_tol=1e-14), g_tol=1e-14)
     


 21.237945 seconds (175.10 M allocations: 9.285 GiB, 4.84% gc time)


Results of Optimization Algorithm
 * Algorithm: Fminbox with L-BFGS
 * Starting Point: [0.5,0.5,0.5,0.5,0.5,0.5,0.5]
 * Minimizer: [-19.579462530534947,62.73979273458934, ...]
 * Minimum: 3.658540e-62
 * Iterations: 1
 * Convergence: true
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 1.27e+02 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = 7.23e+50 |f(x)|
   * |g(x)| ≤ 1.0e-14: true 
     |g(x)| = 1.76e-17 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 249
 * Gradient Calls: 249

In [29]:
using JuMP, Ipopt

INFO: Recompiling stale cache file C:\Users\Max\AppData\Local\JuliaPro-0.6.2.1\pkgs-0.6.2.1\lib\v0.6\Ipopt.ji for module Ipopt.


In [49]:
m = Model(solver=IpoptSolver())
@variable(m, θ[1:7])
@constraint(m, θ[1] <= 1)

θ[1] <= 1

In [50]:
myf(a,b,c,d,e,f,g) = -Qn([a,b,c,d,e,f,g], gn_wrapped)
JuMP.register(m, :myf, 7, myf, autodiff=true)

@NLobjective(m, Min, myf(θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7]))

In [ ]:
solve(m)